In [1]:
import sys

sys.path.append("..")
import numpy as np
from ipywidgets import interact

from calibration.simulator.board import draw_board, gen_checkerboard_grid
from calibration.simulator.helpers import SimulParams
from calibration.simulator.simul import simul_projection
from calibration.simulator.types import SimulOut

%load_ext autoreload
%autoreload 2

In [2]:
SimulParams??

Init signature:
SimulParams(
    R: numpy.ndarray = <factory>,
    t: numpy.ndarray = <factory>,
    lambdas: numpy.ndarray = <factory>,
    camera: calibration.simulator.camera.Camera = <factory>,
) -> None
Source:        
@dataclass
class SimulParams:
    """
    A data class to store the simulation parameters for the projection of 3D points
    onto a 2D image plane.

    Attributes:
        R (np.ndarray):
            A 3x3 orthogonal matrix representing the rotation matrix (default: random).
        t (np.ndarray):
            A 3D vector representing the translation vector (default: random).
        lambdas (np.ndarray):
            A 1D array of radial distortion coefficients (default: random).
        camera (Camera):
            A Camera object representing the camera intrinsics (default: Camera()).
        distortion_center (np.ndarray):
            A 3D vector representing the distortion center
            (default: center of the image).
    """

    R: np.ndarray = field(de

In [13]:
def f(t1, t2, t3, l1, l2):
    params = SimulParams(
        R=np.eye(3), t=np.array([t1, t2, t3]), lambdas=np.array([l1, l2])
    )
    print(params.lambdas)
    X, x, lambdas, R, t = simul_projection(gen_checkerboard_grid(7, 9), params)
    # x = (np.linalg.inv(params.camera.intrinsic_matrix) @ np.c_[x, np.ones(x.shape[0])].T).T[:, :2]
    return draw_board(x, "Board", params.camera.resolution).update_layout(
    # return draw_board(x, "Board").update_layout(
        # autosize=False,
        width=700,
        height=700,
    )

In [18]:
interact(
    f,
    t1=(-6.0, -2.0),
    t2=(-4.0, -2.0),
    t3=(20.0, 100.0),
    l1=(-100, 100),
    l2=(-10, 10, 0.001),
);

interactive(children=(FloatSlider(value=-4.0, description='t1', max=-2.0, min=-6.0), FloatSlider(value=-3.0, d…

In [15]:
10 ** 2

100

In [16]:
1 ** 2

1

In [17]:
0.1 ** 2

0.010000000000000002